<a href="https://colab.research.google.com/github/ngonam2403/VEF-Final-Project_202007/blob/master/Chu%E1%BA%A9n_ho%C3%A1_c%C3%A1c_h%C3%A0m_th%C3%B4ng_d%E1%BB%A5ng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Mục đích

Format chung: 
1. mục đích hàm
2. input (dataframe, list) --> output (number, dataframe, chart)
3. cần library nào để chạy? pandas, numpy, scikitlearn, seaborn

---
Mục đích: để tra cứu & tái sử dụng

---
Cách đặt tên phần tử (Naming Convention): mục đích của việc này là để người khác dễ đọc và mường tượng được code của mình.
thứ tự ưu tiên: mục đích -> loại -> tên riêng để định danh.
- hàm: động từ, ví dụ def create_table() hoặc def plot_chart().
- biến: danh từ
  - dataframe: df_, ví dụ: thay vì viết data = pd.read_csv() thì viết df_abc = pd.read_csv(). nếu dataframe là một param trong hàm thì nên viết def create_table(param_df_xyz).
  - một con số: num_
  - text: str_
    
- lúc dùng thì ghi rõ param1 = , param2 =, param3 = 
- khi hàm có quá nhiều param thì chuyển sang dùng class,
- class: CamelCase

- tham khảo: https://www.analyticsvidhya.com/blog/2020/07/python-style-guide/; https://towardsdatascience.com/data-scientists-your-variable-names-are-awful-heres-how-to-fix-them-89053d2855be
---
flow chạy, pipeline.


In [ ]:
# Don't do this
temp = get_house_price_in_usd(house_sqft, house_room_count)
final_value = temp * usd_to_aud_conversion_rate
# Do this instead
house_price_in_usd = get_house_price_in_usd(house_sqft, 
                                            house_room_count)
house_price_in_aud = house_price_in_usd * usd_to_aud_conversion_rate

In [ ]:
for row_index in range(row_count):
    for column_index in range(column_count):
        ....

The benefits of adopting standards are that they let you make a single global decision instead of many local ones.

# 1.Các hàm xử lý missing data

In [ ]:
"""
FUNCTION: CHECK MISSING DATA
input: a dataframe MxN --> output: a dataframe Nx2
need: pandas
"""
def create_table_missing_values(df):
    mis_val = df.isnull().sum() #total missing values #input: df MxN -> output: df Nx2
    mis_val_percent = 100 * df.isnull().sum() / len(df) #percentage of missing values #input: df MxN -> output: df Nx2
    mis_val_table = pd.concat( #join df mis_val and df mis_val_percent
        [mis_val, mis_val_percent], 
        axis=1
        ) 
    mis_val_table_ren_columns = mis_val_table.rename( #rename the columns
        columns = {
            0 : 'Missing values', 
            1 : '% of Total Values'
            }
        )
    mis_val_table_ren_columns = mis_val_table_ren_columns[ #sort the table by percentage of missing descending
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1) #exclude those has 0% missing.
    
    print("Your selected dataframe has ")    
    return mis_val_table_ren_columns

# 2.Các hàm vẽ biểu đồ

## a. KDE

In [ ]:
"""FUNCTION: Kernel Density Estimation plot
input: a dataframe --> output: a plot
need: pandas, seaborn, matplotlib
"""
def create_plot_kdeplot(df, target_col_name, x_variable):

    plt.figure(figsize = (10, 8))

    # # KDE plot of loans that were repaid on time
    # sns.kdeplot(
    #     app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, 
    #     label = 'target == 0')

    # # KDE plot of loans which were not repaid on time
    # sns.kdeplot(
    #     app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, 
    #     label = 'target == 1')

    for i in target_values:
        sns.kdeplot(
            df.loc[df[target_col_name]== i, x_variable],
            label = 'target == {}'.format(i)
            )

    # Labeling of plot
    plt.xlabel('Age (years)'); 
    plt.ylabel('Density'); 
    plt.title('Distribution of Ages')
  

## b. Phân phối

In [ ]:
"""FUNCTION: plot stats https://www.kaggle.com/gpreda/home-credit-default-risk-extensive-eda
input: dataframe --> output: a plot
need pandas, matplotlib, seaborn.
"""
# def f_plot_stats(feature, label_rotation=False, horizontal_layout=True):


""" FUNCTION: plot distribution """
def create_plot_one_distribution(feature, color):
    plt.figure(figsize=(10,6)) #create a figure
    plt.title("Distribution of %s" % feature) #create a title for chart.
    sns.distplot( #
        application_train[feature].dropna(),
        color=color, 
        kde=True,
        bins=100)
    plt.show()   


""" FUNCTION: plot multiple features distribution"""
def create_plot_multiple_distribution_comp(column_list,nrow=2):
    
    i = 0
    t1 = application_train.loc[application_train['TARGET'] != 0]
    t0 = application_train.loc[application_train['TARGET'] == 0]

    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(nrow,2,figsize=(12,6*nrow))

    for feature in var:
        i += 1
        plt.subplot(nrow,2,i)
        sns.kdeplot(t1[feature], bw=0.5,label="TARGET = 1")
        sns.kdeplot(t0[feature], bw=0.5,label="TARGET = 0")
        plt.ylabel('Density plot', fontsize=12)
        plt.xlabel(feature, fontsize=12)
        locs, labels = plt.xticks()
        plt.tick_params(axis='both', which='major', labelsize=12)
    plt.show()

IndentationError: ignored

## c. Histogram

In [ ]:
"""FUNCTION: plot histogram"""
def create_plot_histogram(df,col_name,titleName,xlabelName):
    df[col_name].plot.hist(title = titleName)
    plt.xlabel(xlabelName)
"""example
plot_histogram(df = data,
               col_name = 'DAYS_EMPLOYED',
               titleName = 'Days Employment Histogram',
               xlabelName = 'Days Employment')
"""               

# 3.Các hàm về correlation

In [ ]:
"""FUNTION: top 3 correlation with Target and bottom 3 correlation
input: a dataframe MxN --> output: a dataframe Nx2
need: pandas
"""
def create_table_top_corr(df, target_col_name):
    correlations = df.corr()[target_col_name].sort_values()

    print('Most Positive Correlations:\n', correlations.tail(3))
    print('\nMost Negative Correlations:\n', correlations.head(3))



# 4.Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression() 
"""
create a model.
type(log_reg)
<class 'sklearn.linear_model._logistic.LogisticRegression'>
"""

# Load data
# Cleansing & Transformation pipeline

# Split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(
                                          X,
                                          y,
                                          test_size=0.25,
                                          random_state=0)


# Fit
log_reg.fit(X, y)
"""
'fit' is a method of class 'linear_model'. it receives 
dataframe X and vector y, as input, 
and return self. 
after we run .fit, we can run .predict
"""

# Predict
log_reg.predict(X_test) #log_reg.predict([[1.7], [1.5]])
"""
'predict' is a method.
it receive dataframe X as input, 
and return 'Predicted class label per sample'.
"""

# Evaluation
# 1.Confustion Matrix
from sklearn import metrics
df_confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
df_confusion_matrix

# 2.Accuracy-Precision-Recall, on test set
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

# 3.F1-score
from sklearn.metrics import f1_score
f1_score(
      y_test, 
      y_pred, 
      average='weighted')
--0.9731332318832974


# 6.Các hàm chưa phân loại

## Chọn, select data từ một dataframe

In [ ]:
""" select from where [condition] """

# Thay vì viết
df_abc = application_train[application_train.TARGET==1]
# Nên viết
df_abc = application_train.loc[application_train.TARGET==1]